<a href="https://colab.research.google.com/github/moshimoshisama/Expedia_Analysis/blob/main/1_preliminary_analysis_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook read input datas as pandas dataframe, in order to have preliminary explore on the data. At the end of this notebook, I realized that regarding to ram limit, we can't perform same analysis for huge unbooked records. Therefore I decided to transfer everything into Pyspark.

# **Part1: data readin**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/expedia/source/

/content/drive/MyDrive/expedia/source


In [ ]:
#!unzip "/content/drive/MyDrive/expedia/source/main.zip" -d "/content/drive/MyDrive/expedia/source/"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb

In [ ]:
main = pd.read_csv("/content/drive/MyDrive/expedia/source/main.tsv",sep="\t")

In [ ]:
# create extra columns for analysis
date_list = ['search_timestamp','checkin_date','checkout_date']
main['search_timestamp'] = pd.to_datetime(main['search_timestamp'], errors='coerce',yearfirst=True)
main['year'] = main['search_timestamp'].dt.year
main['month'] = main['search_timestamp'].dt.month

main['checkin_date'] = pd.to_datetime(main['checkin_date'], errors='coerce',yearfirst=True)
main['checkout_date'] = pd.to_datetime(main['checkout_date'], errors='coerce',yearfirst=True)
# Length of stay
main['len_of_stay'] = main['checkout_date'] - main['checkin_date']


In [ ]:
# preview of data
main.head()

,user_id,search_id,search_timestamp,point_of_sale,geo_location_country,is_mobile,destination_id,checkin_date,checkout_date,adult_count,child_count,infant_count,room_count,sort_type,applied_filters,impressions,year,month,len_of_stay
0,001bbff3030c95740d4c55bb72e823e9,541933a931216d270f53277c2de068d0,2021-07-02 19:38:00+00:00,2,1,False,898,2021-07-04,2021-07-05,2,0,0,1,RECOMMENDED,NaN,"1,5695234,1,4.0,250.0,2.0,1,0,3,0,0|2,1734546,...",2021,7,1 days
1,002ba2a7c74dbffc32e4568af230d49a,25b8ebecb15b8f3edb5eed36a1f699b4,2021-06-23 03:39:00+00:00,1,1,False,50,2021-09-10,2021-09-14,2,0,0,1,PRICE ASCENDING,NaN,"1,5151135,0,null,null,null,1,1,1,0,0",2021,6,4 days
2,00c5152ed6ef5b937e8320a260bde084,349c86ff1be41553878e01988ca209d1,2021-06-10 00:08:00+00:00,1,1,False,49,2021-07-07,2021-07-12,2,0,0,1,RECOMMENDED,NaN,"1,1624072,0,4.0,1250.0,4.0,1,1,2,1,0|2,5170759...",2021,6,5 days
3,00d5e2bca7962f67d5695e7e7b92feb8,d3b41f5d906b3f5e56bcafd4a7428634,2021-07-16 01:08:00+00:00,12,1,False,13,2021-08-25,2021-08-28,2,0,0,1,RECOMMENDED,NaN,"1,2391667,1,4.0,1625.0,5.0,0,0,5,0,0|2,4930059...",2021,7,3 days
4,013f92616fe583c4ba0c3919b69cf3d9,a359fd984e5ae5be64af5226dd1e9fe2,2021-07-07 20:49:00+00:00,1,1,False,206,2021-08-30,2021-09-04,1,0,0,1,RECOMMENDED,VIPONLY:FALSE,"1,6714720,1,5.0,225.0,3.0,1,0,5,0,0|2,5593294,...",2021,7,5 days


# **Part2: Explore Data by finding the summaries**

In [ ]:
# Summary of number variables
print ("Number Variables")
main.describe(include=['number'])

'''
out of all research (mean) :
adult_count: 2 
child_count:0.25 
infant_count:0.002 
length of stay: 3
'''

Number Variables


,point_of_sale,geo_location_country,destination_id,adult_count,child_count,infant_count,room_count,year,month,len_of_stay
count,2.577809e+06,2.577809e+06,2.577809e+06,2.577809e+06,2.577809e+06,2.577809e+06,2.577809e+06,2577809.0,2.577809e+06,2577608
mean,4.618789e+00,4.344100e+00,3.832981e+03,2.042396e+00,2.582461e-01,2.262774e-03,1.007956e+00,2021.0,6.331115e+00,3 days 06:38:12.964952616
std,8.851569e+00,1.126134e+01,9.376390e+03,9.691320e-01,7.359578e-01,4.885921e-02,9.953489e-02,0.0,4.706143e-01,3 days 23:35:38.711601209
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2021.0,6.000000e+00,0 days 00:00:00
25%,1.000000e+00,1.000000e+00,6.900000e+01,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2021.0,6.000000e+00,1 days 00:00:00
50%,1.000000e+00,1.000000e+00,4.510000e+02,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2021.0,6.000000e+00,2 days 00:00:00
75%,4.000000e+00,2.000000e+00,2.611000e+03,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2021.0,7.000000e+00,4 days 00:00:00
max,1.790000e+02,2.200000e+02,7.102600e+04,7.000000e+01,2.300000e+01,3.000000e+00,6.000000e+00,2021.0,7.000000e+00,4021 days 00:00:00


In [ ]:
# Summary of number variables
print ("DateTime Variables")
main.describe(include=['datetime'])


DateTime Variables


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,checkin_date,checkout_date
count,2577608,2577608
unique,552,558
top,2021-07-02 00:00:00,2021-07-05 00:00:00
freq,62423,60387
first,2019-11-04 00:00:00,2019-11-08 00:00:00
last,2022-12-07 00:00:00,2032-07-20 00:00:00


In [ ]:
# Most frequently occurance of each variable (mode)
main.mode()

'''
Most frequently occurace: 
Top checkin date:		 2021-07-02
Top checkout date:		 2021-07-05
is_mobile:				 false
adult:					 2
child:					 0
infant:					 0
room_count :			 1
sort_type:				 Recommended
length of stay:			 1 day
'''


,user_id,search_id,search_timestamp,point_of_sale,geo_location_country,is_mobile,destination_id,checkin_date,checkout_date,adult_count,child_count,infant_count,room_count,sort_type,applied_filters,impressions,year,month,len_of_stay
0,b171f2478d13920969b92b430f8595ac,2dffe849a2e45536cd71f97d235586c0,2021-06-15 01:47:00,1,1,False,2,2021-07-02,2021-07-05,2,0,0,1,RECOMMENDED,VIPONLY:FALSE,"1,4853142,0,null,null,null,1,1,1,0,0",2021,6,1 days


# **Part3: Filter out the booked result and merge with amenities to see the percentage of availability of each amenity.**

In [ ]:
impr = main[['user_id','search_id','impressions']]

In [ ]:
user_id = impr['user_id'].tolist()
search_id = impr['search_id'].tolist()
impressions = impr['impressions'].tolist()

In [ ]:
temp_impression = []
for i in range(0, len(user_id)):
    temp_impr = impressions[i]
    temp_impr = temp_impr.split('|')
    for k in range(0, len(temp_impr)):
        temp_attr = temp_impr[k].split(',')
        temp_list = [user_id[0],search_id[0]]
        for j in range(0,len(temp_attr)):
            temp_list.append(temp_attr[j])
        temp_impression.append(temp_list)

In [ ]:
impr_col = ['user_id',
            'search_id',
            'rank',
            'prop_id',
            'is_travel_ad',
            'review_rating',
            'review_count',
            'star_rating',
            'is_free_cancellation',
            'is_drr',
            'price_bucket',
            'num_clicks',
            'is_trans']

impr_df = pd.DataFrame(temp_impression, columns = impr_col)

In [ ]:
# filter out transaction records
impr_df = impr_df[impr_df.is_trans==1]

In [ ]:
temp_impression[:10]

[['4', '4189323', '0', '4.0', '1025.0', '3.0', '0', '1', '4', '3', '1'],
 ['1', '1656386', '0', '4.0', '1025.0', '3.0', '0', '0', '2', '1', '1'],
 ['1', '6172537', '0', '4.0', '1025.0', '4.0', '1', '1', '3', '1', '1'],
 ['1', '6061219', '0', '4.0', '9975.0', '4.0', '1', '1', '1', '1', '1'],
 ['16', '628627', '0', '5.0', '1025.0', '3.0', '0', '0', '5', '0', '1'],
 ['8', '6525789', '0', '4.0', '1025.0', '3.0', '1', '0', '1', '1', '1'],
 ['12', '4979082', '0', '4.0', '1000.0', '3.0', '1', '0', '1', '1', '1'],
 ['2', '159898', '0', '4.0', '1025.0', '3.0', '1', '1', '1', '1', '1'],
 ['3', '3061517', '0', '4.0', '1475.0', '3.0', '1', '1', '1', '1', '1'],
 ['10', '338525', '0', '4.0', '175.0', '3.0', '1', '1', '4', '1', '1']]

In [ ]:
# preview of impression data
impr_df.head()

,rank,prop_id,is_travel_ad,review_rating,review_count,star_rating,is_free_cancellation,is_drr,price_bucket,num_clicks,is_trans
0,4,4189323,0,4.0,1025.0,3.0,0,1,4,3,1
1,1,1656386,0,4.0,1025.0,3.0,0,0,2,1,1
2,1,6172537,0,4.0,1025.0,4.0,1,1,3,1,1
3,1,6061219,0,4.0,9975.0,4.0,1,1,1,1,1
4,16,628627,0,5.0,1025.0,3.0,0,0,5,0,1


In [ ]:
amenities = pd.read_csv("amenities.tsv",sep="\t")

In [ ]:
amenities_bool_list = ['AirConditioning',
                       'AirportTransfer',
                       'Bar',
                       'FreeAirportTransportation',
                       'FreeBreakfast',
                       'FreeParking',
                       'FreeWiFi',
                       'Gym',
                       'HighSpeedInternet',
                       'HotTub',
                       'LaundryFacility',
                       'Parking',
                       'PetsAllowed',
                       'PrivatePool',
                       'SpaServices',
                       'SwimmingPool',
                       'WasherDryer',
                       'WiFi']

amenities[amenities_bool_list] = amenities[amenities_bool_list].astype(np.bool)

In [ ]:
amenities.head()

,prop_id,AirConditioning,AirportTransfer,Bar,FreeAirportTransportation,FreeBreakfast,FreeParking,FreeWiFi,Gym,HighSpeedInternet,HotTub,LaundryFacility,Parking,PetsAllowed,PrivatePool,SpaServices,SwimmingPool,WasherDryer,WiFi
0,3,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
1,5,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False
2,11,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
4,23,True,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False


In [ ]:
amenities['prop_id'] = amenities['prop_id'].astype(np.int64)
impr_df['prop_id'] = impr_df['prop_id'].astype(np.int64)

In [ ]:
# merge on prop id
result = impr_df.merge(amenities, on='prop_id', how='left')

In [ ]:
result.head()

,rank,prop_id,is_travel_ad,review_rating,review_count,star_rating,is_free_cancellation,is_drr,price_bucket,num_clicks,is_trans,AirConditioning,AirportTransfer,Bar,FreeAirportTransportation,FreeBreakfast,FreeParking,FreeWiFi,Gym,HighSpeedInternet,HotTub,LaundryFacility,Parking,PetsAllowed,PrivatePool,SpaServices,SwimmingPool,WasherDryer,WiFi
0,4,4189323,0,4.0,1025.0,3.0,0,1,4,3,1,False,False,True,True,False,True,True,True,False,True,True,False,True,False,False,True,False,False
1,1,1656386,0,4.0,1025.0,3.0,0,0,2,1,1,False,False,False,False,True,True,True,True,True,True,True,False,False,False,False,True,False,False
2,1,6172537,0,4.0,1025.0,4.0,1,1,3,1,1,False,False,True,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False
3,1,6061219,0,4.0,9975.0,4.0,1,1,1,1,1,False,False,True,False,False,True,True,True,False,True,True,False,True,False,True,False,False,True
4,16,628627,0,5.0,1025.0,3.0,0,0,5,0,1,False,False,False,False,True,True,True,True,False,True,True,False,True,False,False,True,False,False


In [ ]:
# Most frequently occurance of each variable (mode)
result.mode()
'''
rank:						 1
is_travel_ad :				 0
review_rating: 				4.0
star_rating: 				 3
is_free_cancellation:		 1
has a discount price:		 1
price_bucket :				 1 - lower price
num_click : 				 1
'''

,rank,prop_id,is_travel_ad,review_rating,review_count,star_rating,is_free_cancellation,is_drr,price_bucket,num_clicks,is_trans,AirConditioning,AirportTransfer,Bar,FreeAirportTransportation,FreeBreakfast,FreeParking,FreeWiFi,Gym,HighSpeedInternet,HotTub,LaundryFacility,Parking,PetsAllowed,PrivatePool,SpaServices,SwimmingPool,WasherDryer,WiFi
0,1,5448913,0,4.0,1025.0,3.0,1,1,1,1,1,False,False,True,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False


In [ ]:
# all booked records (113231)
result_length = len(result)

In [ ]:
# count number of available amenity for later percentage calculation
AirConditioningTrue = result[result.AirConditioning == True].count()
AirportTransferTrue = result[result.AirportTransfer == True].count()
BarTrue = result[result.Bar == True].count()
FreeAirportTransportationTrue = result[result.FreeAirportTransportation == True].count()
FreeBreakfastTrue = result[result.FreeBreakfast == True].count()
FreeParkingTrue = result[result.FreeParking == True].count()
FreeWiFiTrue = result[result.FreeWiFi == True].count()
GymTrue = result[result.Gym == True].count()
HighSpeedInternetTrue = result[result.HighSpeedInternet == True].count()
HotTubTrue = result[result.HotTub == True].count()
LaundryFacilityTrue = result[result.LaundryFacility == True].count()
ParkingTrue = result[result.Parking == True].count()
PrivatePoolTrue = result[result.PrivatePool == True].count()
SpaServicesTrue = result[result.SpaServices == True].count()
SwimmingPoolTrue = result[result.SwimmingPool == True].count()
WasherDryerTrue = result[result.WasherDryer == True].count()
WiFiTrue = result[result.WiFi == True].count()



In [ ]:
# calculate the persentage of each amenity
AirConditioningResult = AirConditioningTrue['AirConditioning']/result_length * 100
AirportTransferResult = AirportTransferTrue['AirportTransfer']/result_length * 100
BarResult = BarTrue['Bar']/result_length * 100
FreeAirportTransportationResult = FreeAirportTransportationTrue['FreeAirportTransportation']/result_length * 100
FreeBreakfastResult = FreeBreakfastTrue['FreeBreakfast']/result_length * 100
FreeParkingResult = FreeParkingTrue['FreeParking']/result_length * 100
FreeWiFiResult = FreeWiFiTrue['FreeWiFi']/result_length * 100
GymResult = GymTrue['Gym']/result_length * 100
HighSpeedInternetResult = HighSpeedInternetTrue['HighSpeedInternet']/result_length * 100
HotTubResult = HotTubTrue['HotTub']/result_length * 100
LaundryFacilityResult = LaundryFacilityTrue['LaundryFacility']/result_length * 100
ParkingResult = ParkingTrue['Parking']/result_length * 100
PrivatePoolResult = PrivatePoolTrue['PrivatePool']/result_length * 100
SpaServicesResult = SpaServicesTrue['SpaServices']/result_length * 100
SwimmingPoolResult = SwimmingPoolTrue['SwimmingPool']/result_length * 100
WasherDryerResult = WasherDryerTrue['WasherDryer']/result_length * 100
WiFiResult = WiFiTrue['WiFi']/result_length * 100


In [ ]:
print("AirConditioningResult", AirConditioningResult)
print("AirportTransferResult", AirportTransferResult)
print("BarResult",BarResult)
print("FreeAirportTransportationResult",FreeAirportTransportationResult)
print("FreeBreakfastResult",FreeBreakfastResult)
print("FreeParkingResult",FreeParkingResult) 
print("FreeWiFiResult",FreeWiFiResult) 
print("GymResult",GymResult) 
print("HighSpeedInternetResult",HighSpeedInternetResult) 
print("HotTubResult",HotTubResult) 
print("LaundryFacilityResult",LaundryFacilityResult)
print("ParkingResult",ParkingResult) 
print("PrivatePoolResult",PrivatePoolResult) 
print("SpaServicesResult",SpaServicesResult) 
print("SwimmingPoolResult",SwimmingPoolResult) 
print("WasherDryerResult",WasherDryerResult) 
print("WiFiResult",WiFiResult)

AirConditioningResult 42.01322959260273
AirportTransferResult 5.666292799675
BarResult 55.13861045120153
FreeAirportTransportationResult 8.151477952150913
FreeBreakfastResult 31.72982663758158
FreeParkingResult 58.59702731584107
FreeWiFiResult 92.91713400040625
GymResult 65.33369836882126
HighSpeedInternetResult 27.492471143061533
HotTubResult 22.520334537361677
LaundryFacilityResult 82.59840503042453
ParkingResult 35.24034937428796
PrivatePoolResult 0.12364105236198568
SpaServicesResult 18.33861751640452
SwimmingPoolResult 47.89854368503325
WasherDryerResult 2.1063136420238275
WiFiResult 10.188905865001633


In [ ]:
'''
FreeWiFi: 					 T 92.91%
LaundryFacility:			 T 82.60%
Gym: 						 T 65.33%
FreeParking: 				 T 58.6%
Bar: 						 T 55.14%
SwimmingPool:				 F 47.9%
AirConditioning: 			 F 42.01%
Parking:					 F 35.24%
FreeBreakfast: 				 F 31.73%
HighSpeedInternet:			 F 27.49%
HotTub:						 F 22.52%
SpaServices:				 F 18.34%
WiFi:						 F 10.19%
FreeAirportTransportation:   F 8.15%
AirportTransfer: 			 F 5.67%
WasherDryer	:				 F 2.1%
PrivatePool:				 F 0.12%
'''